# 1. Pre-processing

#### 1.1. Text Pre-processing

The dataset is already lowercased and lacks punctuation. We will tokenize the text and remove stopwords, as well as apply lemmatization to the tokens.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import gensim
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')

from sklearn.feature_extraction import text

#words to keep: no couldnt cry not cant cannot nor except nobody off but serious enough nothing alone down only without


data = pd.read_json('./data/data.jsonl', lines=True)


def pre_process_data(dataset):
    #tokenize
    dataset['text'] = dataset['text'].apply(nltk.word_tokenize)

    #remove stop words
    my_stop_words = text.ENGLISH_STOP_WORDS
    #print(my_stop_words)
    words_to_keep = frozenset(['no', 'couldnt', 'cry', 'not', 'cant', 'cannot', 'nor', 'except', 'nobody', 'off', 'but', 'serious', 'enough', 'nothing', 'alone', 'down', 'only', 'without','hereby'])
    my_stop_words = my_stop_words - words_to_keep
    
    dataset['text'] = dataset['text'].apply(lambda x: [word for word in x if word not in my_stop_words])

    #lemmatize
    lemmatizer = WordNetLemmatizer()
    dataset['text'] = dataset['text'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])
    
    return dataset

data = pre_process_data(data)
data.head()

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\inesc\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\inesc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\inesc\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,text,label
0,"[feel, awful, s, job, position, succeed, just,...",0
1,"[im, alone, feel, awful]",0
2,"[ive, probably, mentioned, but, really, feel, ...",1
3,"[feeling, little, low, day]",0
4,"[beleive, sensitive, people, feeling, tend, co...",2


In [2]:
test_data = pd.read_json('./data/test.jsonl', lines=True)
train_data = pd.read_json('./data/train.jsonl', lines=True)
validation_data = pd.read_json('./data/validation.jsonl', lines=True)

test_data = pre_process_data(test_data)
train_data = pre_process_data(train_data)
validation_data = pre_process_data(validation_data)
print(test_data['text'][13])

print(test_data["text"])

['just', 'feel', 'extremely', 'comfortable', 'group', 'people', 'dont', 'need', 'hide']
0        [im, feeling, rotten, im, not, ambitious, right]
1                      [im, updating, blog, feel, shitty]
2       [make, separate, don, t, want, feel, like, m, ...
3       [left, bouquet, red, yellow, tulip, arm, feeli...
4                            [feeling, little, vain, did]
                              ...                        
1995    [just, feeling, like, unkind, doing, wrong, th...
1996    [im, feeling, little, cranky, negative, doctor...
1997    [feel, useful, people, give, great, feeling, a...
1998    [im, feeling, comfortable, derby, feel, start,...
1999    [feel, weird, meet, w, people, text, but, like...
Name: text, Length: 2000, dtype: object


# 2. Vectorization
(secção possívelmente temporária, mas queria experimentar as cenas de tf_idf depois do pré-processamento) -- matos
não acho que convenha ser temporaria, dado que efetivamente melhora os resultados ihihihi, e é uma prática comum e recomendada pelo que estivemos a ver -- ines


### TF-IDF

#### Hypertunning for tf-idf

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer




def tfidfVectorizer(data, train_data, validation_data, test_data):
    vectorizer = TfidfVectorizer(stop_words=list(my_stop_words), ngram_range=(1,2), max_features=40000)
    # good idea to use two-grams??
    # print(X.shape)

    x_train = vectorizer.fit_transform(train_data['text'].apply(lambda x: ' '.join(x)))

    x_val = vectorizer.transform(validation_data['text'].apply(lambda x: ' '.join(x)))
    x_test = vectorizer.transform(test_data['text'].apply(lambda x: ' '.join(x)))

    y_train = train_data['label']
    y_val = validation_data['label']
    y_test = test_data['label']
    
    # print(vectorizer.get_feature_names_out())

    return x_train, x_val, x_test, y_train, y_val, y_test



### BOW model
Count vectorizer which is an implementation of the BOW model.

The disadvantage of the BOW model is it does not consider the sequence of words, and as language does involve sequence and context, sometimes the BOW model might not be a good fit for the best-case scenario.

In [4]:
from sklearn.feature_extraction.text import CountVectorizer

def countVectorizer(data, train_data, validation_data, test_data):
    # Initialize CountVectorizer
    vectorizer = CountVectorizer(stop_words=my_stop_words, ngram_range=(1,2), max_features=40000)

    x_train= vectorizer.fit_transform(train_data['text'].apply(lambda x: ' '.join(x)))

    x_val = vectorizer.transform(validation_data['text'].apply(lambda x: ' '.join(x)))
    x_test = vectorizer.transform(test_data['text'].apply(lambda x: ' '.join(x)))

    y_train = train_data['label']
    y_val = validation_data['label']
    y_test = test_data['label']

    return x_train, x_val, x_test, y_train, y_val, y_test



### Doc 2 Vec

In [5]:
# import optuna
# from gensim.models import Doc2Vec
# from gensim.models.doc2vec import TaggedDocument
# from sklearn.metrics import accuracy_score
# from sklearn.linear_model import LogisticRegression
# from sklearn.model_selection import train_test_split
# import numpy as np

# y_train = train_data['label']
# y_val = validation_data['label']

# def objective(trial):
#     vector_size = trial.suggest_int("vector_size", 50, 300)
#     window = trial.suggest_int("window", 3, 15)
#     min_count = trial.suggest_int("min_count", 1, 10)
#     epochs = trial.suggest_int("epochs", 10, 50)
    
#     # Train Doc2Vec model
#     tagged_data = [TaggedDocument(words=doc, tags=[str(label)]) for doc, label in zip(train_data['text'], train_data['label'])]
#     doc2vec_model = Doc2Vec(vector_size=vector_size, window=window, min_count=min_count, epochs=epochs)
#     doc2vec_model.build_vocab(tagged_data)
#     doc2vec_model.train(tagged_data, total_examples=doc2vec_model.corpus_count, epochs=doc2vec_model.epochs)
    
#     # Prepare document vectors for training and test sets
#     X_train_vecs  = [doc2vec_model.infer_vector(doc) for doc in train_data['text']]
#     X_val_vecs = [doc2vec_model.infer_vector(doc) for doc in validation_data['text']]
    
#     # Train Logistic Regression classifier
#     classifier = LogisticRegression(max_iter=1000)
#     classifier.fit(X_train_vecs, y_train)
    
#     # Evaluate on test set
#     y_pred = classifier.predict(X_val_vecs)
#     accuracy = accuracy_score(y_val, y_pred)
    
#     return accuracy

# # Define the study object and optimize the objective function
# study = optuna.create_study(direction="maximize")
# study.optimize(objective, n_trials=100)

# # Print the best hyperparameters found
# best_params = study.best_params
# print("Best hyperparameters:", best_params)

# # Train the final model with the best hyperparameters
# best_vector_size = best_params["vector_size"]
# best_window = best_params["window"]
# best_min_count = best_params["min_count"]
# best_epochs = best_params["epochs"]

# # Train Doc2Vec model with best hyperparameters
# tagged_data = [TaggedDocument(words=doc.split(), tags=[i]) for i, doc in enumerate(train_data["text"])]
# best_doc2vec_model = Doc2Vec(vector_size=best_vector_size, window=best_window, min_count=best_min_count, epochs=best_epochs)
# best_doc2vec_model.build_vocab(tagged_data)
# best_doc2vec_model.train(tagged_data, total_examples=best_doc2vec_model.corpus_count, epochs=best_doc2vec_model.epochs)

# # Prepare document vectors for training and test sets with the best Doc2Vec model
# X_train_vecs  = [best_doc2vec_model.infer_vector(doc) for doc in train_data['text']]
# X_val_vecs = [best_doc2vec_model.infer_vector(doc) for doc in validation_data['text']]

# # Train Logistic Regression classifier on the final Doc2Vec vectors
# final_classifier = LogisticRegression(max_iter=1000)
# final_classifier.fit(X_train_vecs, y_train)

# # Evaluate on test set
# final_accuracy = final_classifier.score(X_val_vecs, y_val)
# print("Final accuracy on test set with the best model:", final_accuracy)


In [6]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn import utils

def train_doc2vec(train_data):
    max_epochs = 16
    vec_size = 88
    alpha = 0.025
    window = 4
    min_count = 7

    tagged_data = [TaggedDocument(words=doc, tags=[str(label)]) for doc, label in zip(train_data['text'], train_data['label'])]

    # antes tinha workers definidos
    model = Doc2Vec(vector_size=vec_size, window=window, min_count=min_count, epochs=max_epochs)
    
    model.build_vocab(tagged_data)


    model.train(tagged_data, total_examples=model.corpus_count, epochs=50)


    model.save("d2v_best_stopwords.model")
    print("Model Saved")

    return model

train_doc2vec(train_data)

Model Saved


In [7]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn import utils
from gensim.models.doc2vec import Doc2Vec
from sklearn.decomposition import PCA

def doc2vec(data, train_data, validation_data, test_data):

    # training the doc2vec model
    #model = train_doc2vec(train_data)
    model = Doc2Vec.load("d2v_best_stopwords.model")
    
    x_train = [model.infer_vector(doc) for doc in train_data['text']]
    x_val = [model.infer_vector(doc) for doc in validation_data['text']]
    x_test = [model.infer_vector(doc) for doc in test_data['text']]

    y_train = train_data['label']
    y_val = validation_data['label']
    y_test = test_data['label']

    return x_train, x_val, x_test, y_train, y_val, y_test

### Word 2 Vec

In [8]:
from gensim.models import Word2Vec
import numpy as np

def lala(model, data):
    vectors = []
    for doc in data['text']:
        document = []
        for word in doc:
            if word in model.wv:
                document.append(model.wv[word])
        vectors.append(document)
    return vectors

# test [d,c,c]  | label
########################
## test 
## [ [] [] [] ] | label

def wordEmbeddingsVectorizer(data):

    data_model = Word2Vec(data["text"], vector_size=3, window=10, min_count=2, workers=10, sg=1)
    
    x_train = lala(data_model, train_data)
    x_val =lala(data_model, validation_data)
    x_test = lala(data_model, test_data)

    y_train = train_data['label']
    y_val = validation_data['label']
    y_test = test_data['label']

    return x_train, x_val, x_test, y_train, y_val, y_test



### Word2Vec - Word embeddings Matos

In [38]:
from gensim.models import Word2Vec
import numpy as np
import gensim.downloader as api


def text_to_vector(embeddings, text, sequence_len):
    
    # split text into tokens
    tokens = text.split()
    
    # convert tokens to embedding vectors, up to sequence_len tokens
    vec = []
    n = 0
    i = 0
    while i < len(tokens) and n < sequence_len:   # while there are tokens and did not reach desired sequence length
        try:
            vec.extend(embeddings.get_vector(tokens[i]))
            n += 1
        except KeyError:
            True   # simply ignore out-of-vocabulary tokens
        finally:
            i += 1
    
    # add blanks up to sequence_len, if needed
    for j in range(sequence_len - n):
        vec.extend(np.zeros(embeddings.vector_size,))
    
    return vec

def text_to_mean_vector(embeddings, text):
     # split text into tokens
    tokens = text.split()

    # convert tokens to embedding vectors, up to sequence_len tokens
    total_vector = []
    i = 0
    while i < len(tokens):   # while there are tokens and did not reach desired sequence length
        try:
            total_vector.append(embeddings.get_vector(tokens[i]))
        except KeyError:
            True   # simply ignore out-of-vocabulary tokens
        finally:
            i += 1

    return np.mean(total_vector, axis=0)

def documents_to_vectors(embeddings, documents, sequence_len):
    vectors = []
    for doc in documents:
        vectors.append(text_to_vector(embeddings, doc, sequence_len))
    return np.array(vectors)


def documents_to_mean_vectors(embeddings, documents):
    vectors = []
    for doc in documents:
        vectors.append(text_to_mean_vector(embeddings, doc))
    return np.array(vectors)

def wordEmbeddingsVectorizer(data):
    
    # use twitter word embeddings
    model_glove_twitter = api.load("glove-twitter-25")
    
    # Initialize Word2Vec
    
    train = train_data['text'].apply(lambda x: ' '.join(x))
    val = validation_data['text'].apply(lambda x: ' '.join(x))
    test = test_data['text'].apply(lambda x: ' '.join(x))
    
    x_train = documents_to_vectors(model_glove_twitter, train, 40)
    x_val = documents_to_vectors(model_glove_twitter, val, 40)
    x_test = documents_to_vectors(model_glove_twitter, test, 40)
    
    print(x_train.shape, x_val.shape, x_test.shape)
    print(x_train[0])
    
    y_train = train_data['label']
    y_val = validation_data['label']
    y_test = test_data['label']

    return x_train, x_val, x_test, y_train, y_val, y_test


def wordEmbeddingsMeanVectorizer(data):

    # use twitter word embeddings
    model_glove_twitter = api.load("glove-twitter-25")

    # Initialize Word2Vec

    train = train_data['text'].apply(lambda x: ' '.join(x))
    val = validation_data['text'].apply(lambda x: ' '.join(x))
    test = test_data['text'].apply(lambda x: ' '.join(x))

    x_train = documents_to_mean_vectors(model_glove_twitter, train)
    x_val = documents_to_mean_vectors(model_glove_twitter, val)
    x_test = documents_to_mean_vectors(model_glove_twitter, test)

    # x_train = np.mean(x_train, axis=1)
    # x_val = np.mean(x_val, axis=1)
    # x_test = np.mean(x_test, axis=1)
    #
    print(x_train.shape, x_val.shape, x_test.shape)
    print(x_train[0])

    y_train = train_data['label']
    y_val = validation_data['label']
    y_test = test_data['label']

    return x_train, x_val, x_test, y_train, y_val, y_test


### Choosing algorithm

In [40]:
# Choose vectorizer (featurizer)
#x_train, x_val, x_test, y_train, y_val, y_test = tfidfVectorizer(data, train_data, validation_data, test_data)
#x_train, x_val, x_test, y_train, y_val, y_test = countVectorizer(data, train_data, validation_data, test_data)
x_train, x_val, x_test, y_train, y_val, y_test = wordEmbeddingsMeanVectorizer(data)
#x_train, x_val, x_test, y_train, y_val, y_test = doc2vec(data, train_data, validation_data, test_data)

#print(x_train)


[==================================================] 100.0% 104.8/104.8MB downloaded
(16000, 25) (2000, 25) (2000, 25)
[-0.35627666  0.37016702  0.28562868 -0.47295332 -0.5652533  -0.24137668
  0.8951466  -0.6888234  -0.95984     0.14970261 -0.6163067   0.84833336
 -3.2683165  -0.29839    -0.04744667  0.45890665  0.69503003 -0.26880667
  0.34184667 -1.4003      0.45056033  0.8567834   0.38757202  0.03085001
 -0.44497335]


### Apply SMOTE

In [41]:
## Apply SMOTE
smote = SMOTE(random_state=4, sampling_strategy='auto', k_neighbors=18)
x_train, y_train = smote.fit_resample(x_train, y_train)

#### Hypertunning number of features (using PCA) for each model

In [11]:
# import optuna
# from sklearn.decomposition import PCA
# from sklearn.linear_model import LogisticRegression
# from sklearn.pipeline import Pipeline
# from sklearn.metrics import accuracy_score
# from sklearn.svm import SVC

# def objective(trial):
#     # Define parameters to search
#     n_components = trial.suggest_int('n_components', 65, 88)
    
#     # Define the pipeline with PCA and a classifier
#     pipeline = Pipeline([
#         ('pca', PCA(n_components=n_components)),
#         ('clf', SVC())
#         #('clf', LogisticRegression())
#     ])

#     # Fit the pipeline
#     pipeline.fit(x_train, y_train)
    
#     # Evaluate on the validation set
#     y_pred = pipeline.predict(x_val)
#     accuracy = accuracy_score(y_val, y_pred)
    
#     return accuracy

# # Optimize hyperparameters
# study = optuna.create_study(direction='maximize')
# study.optimize(objective, n_trials=100)

# # Get the best parameters found
# best_params = study.best_params

# # Get the best model
# best_n_components = best_params['n_components']
# best_model = Pipeline([
#     ('pca', PCA(n_components=best_n_components)),
#     ('clf', )  
# ])
# best_model.fit(x_train, y_train)

# # Use the best model for prediction
# y_pred = best_model.predict(x_test)

# # Evaluate the best model
# accuracy = accuracy_score(y_test, y_pred)
# print("Test Accuracy:", accuracy)



# 3. Model Training

#### 3.1. Model Selection
Aqui também só estava a querer espetar modelos para começar a ver o que dá que ainda não sei que features vão ser usadas:
tf-idf, word embeddings, ???, features mais feitas à mão?


Isto pelos vistos é uma cena, que não implementei (ainda..)

"The validation set uses a subset of the training data to provide an unbiased evaluation of a model. The validation data set contrasts with training and test sets in that it is an intermediate phase used for choosing the best model and optimizing it. It is in this phase that hyperparameter tuning occurs."

Wikipedia:
The basic process of using a validation data set for model selection (as part of training data set, validation data set, and test data set) is:

Since our goal is to find the network having the best performance on new data, the simplest approach to the comparison of different networks is to evaluate the error function using data which is independent of that used for training. Various networks are trained by minimization of an appropriate error function defined with respect to a training data set. The performance of the networks is then compared by evaluating the error function using an independent validation set, and the network having the smallest error with respect to the validation set is selected. This approach is called the hold out method. Since this procedure can itself lead to some overfitting to the validation set, the performance of the selected network should be confirmed by measuring its performance on a third independent set of data called a test set.

An application of this process is in early stopping, where the candidate models are successive iterations of the same network, and training stops when the error on the validation set grows, choosing the previous model (the one with minimum error).

##### 3.1.1. Logistic Regression

##### Model hypertunning

In [15]:
# import optuna
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
# from sklearn.model_selection import train_test_split

# # Define objective function for Optuna
# # Define objective function for Optuna
# def objective(trial):
#     # Define solver
#     solver = trial.suggest_categorical('solver', ['lbfgs', 'liblinear', 'saga'])
    
#     if solver == 'lbfgs':
#         penalty = 'l2'  # LBFGS only supports L2 penalty
#     else:
#         penalty = trial.suggest_categorical('penalty', ['l1', 'l2'])
    
#     C = trial.suggest_loguniform('C', 0.01, 10)
    
#     # Instantiate Logistic Regression classifier with hyperparameters
#     logreg_classifier = LogisticRegression(max_iter=1000, solver=solver, penalty=penalty, C=C)
    
#     # Train classifier
#     logreg_classifier.fit(x_train, y_train)
    
#     # Predict on validation set
#     y_pred = logreg_classifier.predict(x_val)
    
#     # Calculate accuracy score
#     accuracy = accuracy_score(y_val, y_pred)
    
#     return accuracy

# # Create Optuna study
# study = optuna.create_study(direction='maximize')
# study.optimize(objective, n_trials=100)

# # Get best hyperparameters
# best_params = study.best_params

# print(best_params)

[I 2024-04-01 11:41:53,457] A new study created in memory with name: no-name-6469883b-e42d-44e8-9384-d37484e49d1d
C:\Users\inesc\AppData\Local\Temp\ipykernel_13684\506814322.py:17: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform('C', 0.01, 10)
[I 2024-04-01 11:41:55,390] Trial 0 finished with value: 0.808 and parameters: {'solver': 'lbfgs', 'C': 0.08716883426232369}. Best is trial 0 with value: 0.808.
C:\Users\inesc\AppData\Local\Temp\ipykernel_13684\506814322.py:17: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform('C', 0.01, 10)
[I 2024-04-01 11:41:57,715] Trial 1 finished with value: 0.8115 and parameters: {'solver'

{'solver': 'liblinear', 'penalty': 'l2', 'C': 0.03784347512520397}


##### Model Training

In [42]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# pca = PCA(n_components=77)
# x_train = pca.fit_transform(x_train)
# x_val = pca.transform(x_val)
# x_test = pca.transform(x_test)

logreg_classifier = LogisticRegression(solver='liblinear', penalty='l2', C=0.03784347512520397)
logreg_classifier.fit(x_train, y_train)
y_pred = logreg_classifier.predict(x_val)
print(accuracy_score(y_val, y_pred))
print(classification_report(y_val, y_pred))
print(confusion_matrix(y_val, y_pred))

0.3625
              precision    recall  f1-score   support

           0       0.50      0.31      0.39       550
           1       0.68      0.38      0.49       704
           2       0.23      0.48      0.32       178
           3       0.29      0.33      0.31       275
           4       0.27      0.33      0.30       212
           5       0.11      0.41      0.17        81

    accuracy                           0.36      2000
   macro avg       0.35      0.38      0.33      2000
weighted avg       0.47      0.36      0.39      2000

[[173  50  51 124  66  86]
 [ 55 271 176  48  54 100]
 [ 14  24  86  19  15  20]
 [ 62  17  22  91  45  38]
 [ 27  24  24  30  71  36]
 [ 15  12   7   6   8  33]]


##### 3.1.2. Multinomial Naive Bayes

In [ ]:

# mnb_classifier = MultinomialNB()
# mnb_classifier.fit(x_train, y_train)
# y_pred = mnb_classifier.predict(x_val)
# print(accuracy_score(y_val, y_pred))
# print(classification_report(y_val, y_pred))
# print(confusion_matrix(y_val, y_pred))



## SVM


##### Hypertunning

In [20]:
# import optuna
# from sklearn.svm import SVC
# from sklearn.metrics import accuracy_score
# from sklearn.model_selection import train_test_split

# # Define objective function for Optuna
# def objective(trial):
#     # Define kernel
#     kernel = trial.suggest_categorical('kernel', ['linear', 'poly', 'rbf', 'sigmoid'])
    
#     # For 'poly', 'rbf', and 'sigmoid' kernels, suggest degree parameter
#     if kernel in ['poly', 'rbf', 'sigmoid']:
#         degree = trial.suggest_int('degree', 2, 5)
#     else:
#         degree = 3  # Default degree value for kernels that don't use it
    
#     # Instantiate SVM classifier with hyperparameters
#     svm_classifier = SVC(kernel=kernel, degree=degree)
    
#     # Train classifier
#     svm_classifier.fit(x_train, y_train)
    
#     # Predict on validation set
#     y_pred = svm_classifier.predict(x_val)
    
#     # Calculate accuracy score
#     accuracy = accuracy_score(y_val, y_pred)
    
#     return accuracy

# # Create Optuna study
# study = optuna.create_study(direction='maximize')
# study.optimize(objective, n_trials=100)

# # Get best hyperparameters
# best_params = study.best_params

# print(best_params)

# #[I 2024-04-01 12:39:28,014] Trial 36 finished with value: 0.8245 and parameters: {'kernel': 'poly', 'degree': 2}. Best is trial 36 with value: 0.8245.


[I 2024-04-01 13:36:28,710] A new study created in memory with name: no-name-db33fa6c-c7e1-46ac-8938-b0dcabd6890f
[I 2024-04-01 13:36:57,924] Trial 0 finished with value: 0.704 and parameters: {'kernel': 'sigmoid', 'degree': 5}. Best is trial 0 with value: 0.704.
[I 2024-04-01 13:37:26,935] Trial 1 finished with value: 0.704 and parameters: {'kernel': 'sigmoid', 'degree': 2}. Best is trial 0 with value: 0.704.
[I 2024-04-01 13:37:56,615] Trial 2 finished with value: 0.704 and parameters: {'kernel': 'sigmoid', 'degree': 2}. Best is trial 0 with value: 0.704.
[I 2024-04-01 13:38:08,327] Trial 3 finished with value: 0.808 and parameters: {'kernel': 'linear'}. Best is trial 3 with value: 0.808.
[I 2024-04-01 13:38:29,996] Trial 4 finished with value: 0.821 and parameters: {'kernel': 'rbf', 'degree': 5}. Best is trial 4 with value: 0.821.
[I 2024-04-01 13:38:49,239] Trial 5 finished with value: 0.821 and parameters: {'kernel': 'rbf', 'degree': 2}. Best is trial 4 with value: 0.821.
[I 2024-

KeyboardInterrupt: 

### Training

In [21]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

svm_classifier = SVC(kernel='poly', degree=2, random_state=42)
svm_classifier.fit(x_train, y_train)
y_pred = svm_classifier.predict(x_val)

# Evaluate the performance
accuracy = accuracy_score(y_val, y_pred)
report = classification_report(y_val, y_pred)
conf_matrix = confusion_matrix(y_val, y_pred)

# Print the evaluation metrics
print("Accuracy:", accuracy)
print("Classification Report:\n", report)
print("Confusion Matrix:\n", conf_matrix)

Accuracy: 0.8245
Classification Report:
               precision    recall  f1-score   support

           0       0.85      0.88      0.86       550
           1       0.90      0.85      0.88       704
           2       0.71      0.73      0.72       178
           3       0.80      0.79      0.79       275
           4       0.75      0.76      0.75       212
           5       0.61      0.68      0.64        81

    accuracy                           0.82      2000
   macro avg       0.77      0.78      0.78      2000
weighted avg       0.83      0.82      0.83      2000

Confusion Matrix:
 [[486  13   5  22  21   3]
 [ 24 601  39  16  13  11]
 [ 11  23 130   7   5   2]
 [ 28  12   3 216  10   6]
 [ 18   8   4   8 161  13]
 [  7  10   1   2   6  55]]


## KNN

##### Hypertunning

In [22]:
# import optuna
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
# from sklearn.model_selection import train_test_split

# # Define objective function for Optuna
# def objective(trial):
#     # Define hyperparameters to optimize
#     n_neighbors = trial.suggest_int('n_neighbors', 1, 10)
#     weights = trial.suggest_categorical('weights', ['uniform', 'distance'])
#     algorithm = trial.suggest_categorical('algorithm', ['auto', 'ball_tree', 'kd_tree', 'brute'])
    
#     # Instantiate KNN classifier with hyperparameters
#     knn_classifier = KNeighborsClassifier(n_neighbors=n_neighbors, weights=weights, algorithm=algorithm)
    
#     # Train classifier
#     knn_classifier.fit(x_train, y_train)
    
#     # Predict on validation set
#     y_pred = knn_classifier.predict(x_val)
    
#     # Calculate accuracy score
#     accuracy = accuracy_score(y_val, y_pred)
    
#     return accuracy

# # Create Optuna study
# study = optuna.create_study(direction='maximize')
# study.optimize(objective, n_trials=100)

# # Get best hyperparameters
# best_params = study.best_params

# print(best_params)

# # [I 2024-04-01 13:53:46,403] Trial 26 finished with value: 0.701 and parameters: {'n_neighbors': 10, 'weights': 'uniform', 'algorithm': 'auto'}. Best is trial 26 with value: 0.701.

[I 2024-04-01 13:50:59,436] A new study created in memory with name: no-name-9368ba41-2d16-46b7-a9f4-4a1b490a48ce
[I 2024-04-01 13:51:01,097] Trial 0 finished with value: 0.681 and parameters: {'n_neighbors': 4, 'weights': 'distance', 'algorithm': 'brute'}. Best is trial 0 with value: 0.681.
[I 2024-04-01 13:51:24,154] Trial 1 finished with value: 0.69 and parameters: {'n_neighbors': 10, 'weights': 'distance', 'algorithm': 'kd_tree'}. Best is trial 1 with value: 0.69.
[I 2024-04-01 13:51:36,840] Trial 2 finished with value: 0.6735 and parameters: {'n_neighbors': 3, 'weights': 'distance', 'algorithm': 'ball_tree'}. Best is trial 1 with value: 0.69.
[I 2024-04-01 13:51:38,195] Trial 3 finished with value: 0.6825 and parameters: {'n_neighbors': 7, 'weights': 'distance', 'algorithm': 'brute'}. Best is trial 1 with value: 0.69.
[I 2024-04-01 13:51:51,421] Trial 4 finished with value: 0.6925 and parameters: {'n_neighbors': 9, 'weights': 'uniform', 'algorithm': 'ball_tree'}. Best is trial 4 w

{'n_neighbors': 10, 'weights': 'uniform', 'algorithm': 'auto'}


In [32]:
from sklearn.neighbors import KNeighborsClassifier

#knn_classifier = KNeighborsClassifier(n_neighbors=5)
hyperparameters = {'n_neighbors': 10, 'weights': 'uniform', 'algorithm': 'auto'}
knn_classifier = KNeighborsClassifier(**hyperparameters)
knn_classifier.fit(x_train, y_train)
y_pred = knn_classifier.predict(x_val)

# Evaluate the performance
accuracy = accuracy_score(y_val, y_pred)
report = classification_report(y_val, y_pred)
conf_matrix = confusion_matrix(y_val, y_pred)

# Print the evaluation metrics
print("Accuracy:", accuracy)
print("Classification Report:\n", report)
print("Confusion Matrix:\n", conf_matrix)

Accuracy: 0.701
Classification Report:
               precision    recall  f1-score   support

           0       0.82      0.73      0.77       550
           1       0.88      0.69      0.78       704
           2       0.51      0.74      0.60       178
           3       0.62      0.68      0.65       275
           4       0.58      0.65      0.61       212
           5       0.37      0.74      0.50        81

    accuracy                           0.70      2000
   macro avg       0.63      0.70      0.65      2000
weighted avg       0.74      0.70      0.71      2000

Confusion Matrix:
 [[401  18  18  51  37  25]
 [ 38 487  85  37  26  31]
 [  9  17 131   7   8   6]
 [ 23  17  10 186  25  14]
 [ 16   9  10  15 137  25]
 [  4   3   5   4   5  60]]


### Boosting Algorithms

Testing with some boosting algorithms

##### XGBOOST

In [ ]:
import xgboost as xgb
# XGBoost Classifier
xgb_classifier = xgb.XGBClassifier()
xgb_classifier.fit(x_train, y_train)
y_pred_xgb = xgb_classifier.predict(x_val)

# Evaluate the performance for XGBoost
accuracy_xgb = accuracy_score(y_val, y_pred_xgb)
report_xgb = classification_report(y_val, y_pred_xgb)
conf_matrix_xgb = confusion_matrix(y_val, y_pred_xgb)

# Print the evaluation metrics for XGBoost
print("XGBoost Classifier Evaluation:")
print("Accuracy:", accuracy_xgb)
print("Classification Report:\n", report_xgb)
print("Confusion Matrix:\n", conf_matrix_xgb)

XGBoost Classifier Evaluation:
Accuracy: 0.788
Classification Report:
               precision    recall  f1-score   support

           0       0.80      0.87      0.83       550
           1       0.82      0.87      0.85       704
           2       0.71      0.61      0.65       178
           3       0.77      0.68      0.72       275
           4       0.73      0.70      0.71       212
           5       0.71      0.57      0.63        81

    accuracy                           0.79      2000
   macro avg       0.76      0.71      0.73      2000
weighted avg       0.78      0.79      0.78      2000

Confusion Matrix:
 [[476  28   8  20  15   3]
 [ 29 612  30  19  12   2]
 [ 16  46 108   4   3   1]
 [ 33  28   4 186  20   4]
 [ 27  16   2  10 148   9]
 [ 15  12   1   2   5  46]]


##### Lightgbm

In [ ]:
import lightgbm as lgb

lgb_classifier = lgb.LGBMClassifier()
lgb_classifier.fit(x_train, y_train)
y_pred_lgb = lgb_classifier.predict(x_val)

# Evaluate the performance for LightGBM
accuracy_lgb = accuracy_score(y_val, y_pred_lgb)
report_lgb = classification_report(y_val, y_pred_lgb)
conf_matrix_lgb = confusion_matrix(y_val, y_pred_lgb)

# Print the evaluation metrics for LightGBM
print("\nLightGBM Classifier Evaluation:")
print("Accuracy:", accuracy_lgb)
print("Classification Report:\n", report_lgb)
print("Confusion Matrix:\n", conf_matrix_lgb)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.025435 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 22440
[LightGBM] [Info] Number of data points in the train set: 32172, number of used features: 88
[LightGBM] [Info] Start training from score -1.791759
[LightGBM] [Info] Start training from score -1.791759
[LightGBM] [Info] Start training from score -1.791759
[LightGBM] [Info] Start training from score -1.791759
[LightGBM] [Info] Start training from score -1.791759
[LightGBM] [Info] Start training from score -1.791759

LightGBM Classifier Evaluation:
Accuracy: 0.789
Classification Report:
               precision    recall  f1-score   support

           0       0.79      0.86      0.82       550
           1       0.82      0.88      0.85       704
           2       0.71      0.58      0.64       178
           3       0.78      0.68      0.73       275
           4       0.75      0.71      0.7

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

# AdaBoost Classifier
adaboost_classifier = AdaBoostClassifier()
adaboost_classifier.fit(x_train, y_train)
y_pred_adaboost = adaboost_classifier.predict(x_val)

# Evaluate the performance for AdaBoost
accuracy_adaboost = accuracy_score(y_val, y_pred_adaboost)
report_adaboost = classification_report(y_val, y_pred_adaboost)
conf_matrix_adaboost = confusion_matrix(y_val, y_pred_adaboost)

# Print the evaluation metrics for AdaBoost
print("AdaBoost Classifier Evaluation:")
print("Accuracy:", accuracy_adaboost)
print("Classification Report:\n", report_adaboost)
print("Confusion Matrix:\n", conf_matrix_adaboost)


AdaBoost Classifier Evaluation:
Accuracy: 0.6875
Classification Report:
               precision    recall  f1-score   support

           0       0.74      0.74      0.74       550
           1       0.79      0.76      0.77       704
           2       0.56      0.54      0.55       178
           3       0.61      0.52      0.56       275
           4       0.55      0.69      0.62       212
           5       0.44      0.58      0.50        81

    accuracy                           0.69      2000
   macro avg       0.62      0.64      0.63      2000
weighted avg       0.69      0.69      0.69      2000

Confusion Matrix:
 [[407  28  17  37  46  15]
 [ 47 533  47  26  29  22]
 [  9  48  97   9  11   4]
 [ 52  37   8 144  27   7]
 [ 25  13   3  13 147  11]
 [  8  14   1   6   5  47]]


###  Bagging Algorithm

In [ ]:
from sklearn.ensemble import BaggingClassifier, VotingClassifier

model1 = xgb_classifier
model2 = svm_classifier
model3 = logreg_classifier

voting_clf = VotingClassifier(estimators=[
    ('model1', model1), 
    ('model2', model2),
    ('model3', model3)
], voting='soft')
# Fit the ensemble model
voting_clf.fit(x_train, y_train)

# Predict the validation set
y_pred_voting = voting_clf.predict(x_val)

# Evaluate the performance for the ensemble model
accuracy_voting = accuracy_score(y_val, y_pred_voting)
report_voting = classification_report(y_val, y_pred_voting)
conf_matrix_voting = confusion_matrix(y_val, y_pred_voting)

# Print the evaluation metrics for the ensemble model
print("Voting Classifier Evaluation:")
print("Accuracy:", accuracy_voting)
print("Classification Report:\n", report_voting)
print("Confusion Matrix:\n", conf_matrix_voting)


# 5. Model Evaluation

In [43]:
# evaluate with test set
y_pred = logreg_classifier.predict(x_test)
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

y_pred_logreg = logreg_classifier.predict(x_test)

0.3795
              precision    recall  f1-score   support

           0       0.59      0.35      0.44       581
           1       0.67      0.39      0.50       695
           2       0.18      0.44      0.25       159
           3       0.35      0.36      0.35       275
           4       0.30      0.36      0.33       224
           5       0.11      0.52      0.18        66

    accuracy                           0.38      2000
   macro avg       0.37      0.40      0.34      2000
weighted avg       0.50      0.38      0.41      2000

[[201  57  76  96  64  87]
 [ 47 273 181  48  52  94]
 [ 14  29  70  15  10  21]
 [ 42  17  31 100  50  35]
 [ 33  23  25  26  81  36]
 [  3   8   8   4   9  34]]


In [ ]:
# y_pred = mnb_classifier.predict(x_test)
# print(accuracy_score(y_test, y_pred))
# print(classification_report(y_test, y_pred))
# print(confusion_matrix(y_test, y_pred))

In [36]:
y_pred = svm_classifier.predict(x_test)
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))


0.815
              precision    recall  f1-score   support

           0       0.88      0.87      0.88       581
           1       0.89      0.84      0.86       695
           2       0.64      0.74      0.69       159
           3       0.76      0.74      0.75       275
           4       0.77      0.77      0.77       224
           5       0.49      0.71      0.58        66

    accuracy                           0.81      2000
   macro avg       0.74      0.78      0.75      2000
weighted avg       0.82      0.81      0.82      2000

[[504  15   4  31  20   7]
 [ 16 584  57  11  10  17]
 [  6  23 118   7   2   3]
 [ 24  17   4 204  16  10]
 [ 16   7   2  14 173  12]
 [  5  10   0   1   3  47]]


In [35]:
y_pred = xgb_classifier.predict(x_test)
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))


NameError: name 'xgb_classifier' is not defined

In [ ]:
y_pred = lgb_classifier.predict(x_test)
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

0.777
              precision    recall  f1-score   support

           0       0.80      0.88      0.84       581
           1       0.80      0.86      0.83       695
           2       0.68      0.56      0.62       159
           3       0.75      0.63      0.69       275
           4       0.75      0.69      0.72       224
           5       0.59      0.44      0.50        66

    accuracy                           0.78      2000
   macro avg       0.73      0.68      0.70      2000
weighted avg       0.77      0.78      0.77      2000

[[509  33   5  18  13   3]
 [ 31 598  31  14  14   7]
 [ 13  47  89   8   1   1]
 [ 43  41   2 174  13   2]
 [ 27  18   3  14 155   7]
 [ 10  14   0   3  10  29]]


In [ ]:
y_pred = adaboost_classifier.predict(x_test)
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

0.661
              precision    recall  f1-score   support

           0       0.76      0.70      0.73       581
           1       0.77      0.72      0.74       695
           2       0.49      0.55      0.51       159
           3       0.55      0.55      0.55       275
           4       0.55      0.63      0.59       224
           5       0.32      0.47      0.38        66

    accuracy                           0.66      2000
   macro avg       0.57      0.60      0.59      2000
weighted avg       0.67      0.66      0.67      2000

[[409  48  15  52  50   7]
 [ 46 501  59  42  17  30]
 [ 12  36  87   9  11   4]
 [ 39  35  12 152  27  10]
 [ 23  21   5  18 142  15]
 [ 10   9   1   5  10  31]]


In [ ]:
y_pred = knn_classifier.predict(x_test)
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

0.6805
              precision    recall  f1-score   support

           0       0.80      0.74      0.77       581
           1       0.85      0.66      0.75       695
           2       0.44      0.67      0.53       159
           3       0.59      0.63      0.61       275
           4       0.55      0.64      0.60       224
           5       0.35      0.65      0.46        66

    accuracy                           0.68      2000
   macro avg       0.60      0.67      0.62      2000
weighted avg       0.72      0.68      0.69      2000

[[431  27  19  49  38  17]
 [ 38 462  88  42  37  28]
 [  8  21 107   7  10   6]
 [ 33  17  16 174  22  13]
 [ 24  11   9  20 144  16]
 [  3   6   3   2   9  43]]


# 6. Cause of errors


In [ ]:
emotions_to_int = {
    'sadness': 0,
    'joy': 1,
    'love': 2,
    'anger': 3,
    'fear': 4,
    'surprise': 5
}

int_to_emotions = {v: k for k, v in emotions_to_int.items()}


wrong_predictions = y_test[y_test != y_pred_logreg].index
for i, index in enumerate(wrong_predictions):
    if i == 10:
        break
    print(test_data['text'][index])
    print('Real: ', int_to_emotions[test_data['label'][index]])
    print('Pred:', int_to_emotions[y_pred_logreg[index]])
    print('\n')



In [ ]:
print(confusion_matrix(y_test, y_pred_logreg))
print(classification_report(y_test, y_pred_logreg))

In [ ]:
# check which emotions are being confused
conf_matrix = confusion_matrix(y_test, y_pred_logreg)
conf_matrix_copy = conf_matrix.copy()
for i in range(conf_matrix.shape[0]):
    conf_matrix[i, i] = 0 # make the diagonal null, to not eclipse the other values
    
plt.imshow(conf_matrix, cmap='viridis', interpolation='nearest')
# annotate the axes with the emotion names
plt.xticks(range(6), int_to_emotions.values(), rotation=45)
plt.yticks(range(6), int_to_emotions.values())
# add colorbar more to the right
# plt.colorbar()
# legend the axes with predicted and true values
plt.xlabel('Predicted')
plt.ylabel('True')

# add counts in the plot
for i in range(conf_matrix.shape[0]):
    for j in range(conf_matrix.shape[1]):
        if i == j:
            plt.text(j, i, conf_matrix_copy[i, j], ha='center', va='center', color='white')
        else:
            plt.text(j, i, conf_matrix[i, j], ha='center', va='center', color='black')

for i in range(conf_matrix.shape[0]): # this does not count correct predictions
    plt.text(6, i, conf_matrix[i, :].sum(), ha='center', va='center', color='black')
for i in range(conf_matrix.shape[1]):
    plt.text(i, 7, conf_matrix[:, i].sum(), ha='center', va='center', color='black')
plt.show()

The test set appears to be unbalance.
- 'joy' is a lot more mixed with 'love' than the contrary. 
- 'surprise' has a low Precision

In [ ]:
conf_matrix = confusion_matrix(y_test, y_pred_logreg)
conf_matrix = conf_matrix / conf_matrix.sum(axis=1)[:, None] # normalize the confusion matrix
conf_matrix_copy = conf_matrix.copy()
for i in range(conf_matrix.shape[0]):
    conf_matrix[i, i] = 0 # make the diagonal null, to not eclipse the other values
plt.imshow(conf_matrix, cmap='viridis', interpolation='nearest')

plt.xticks(range(6), int_to_emotions.values(), rotation=45)
plt.yticks(range(6), int_to_emotions.values())
plt.colorbar()
plt.xlabel('Predicted')
plt.ylabel('True')

# add counts in the plot
for i in range(conf_matrix.shape[0]):
    for j in range(conf_matrix.shape[1]):
        if i == j:
            plt.text(j, i, f'{conf_matrix_copy[i, j]:.2f}', ha='center', va='center', color='white')
        else:
            plt.text(j, i, f'{conf_matrix[i, j]:.2f}', ha='center', va='center', color='black')

        
plt.title('Percentage of predictions (row sum=1)')
# Rows add to 1
plt.show()



- sadness mixed with joy and anger
- joy mixed with love
- love mixed with joy
- anger mixed with sadness and joy??
- fear mixed with sadness
- surprise mixed with almost everything

# 7. Experiment with user-inputed setences

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction import text

emotions = {
    0: "0. sadness",
    1: "1. joy",
    2: "2. love",
    3: "3. anger",
    4: "4. fear",
    5: "5. surprise"
}

def preprocess_text(phrase):
    # Tokenize
    tokens = nltk.word_tokenize(phrase.lower())

    # Remove stop words
    my_stop_words = set(stopwords.words('english'))
    words_to_keep = frozenset(['no', 'couldnt', 'cry', 'not', 'cant', 'cannot', 'nor', 'except', 'nobody', 'off', 'but', 'serious', 'enough', 'nothing', 'alone', 'down', 'only', 'without','hereby'])
    my_stop_words -= words_to_keep
    tokens = [word for word in tokens if word not in my_stop_words]

    # Lemmatize
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    return tokens

def doc2vec_text(tokens):
    model = Doc2Vec.load("d2v_best_stopwords.model")
    return model.infer_vector(tokens)

def classify_emotion(number):
    if number==0:
        return "0. sadness"
    if number==1:
        return "1. joy"
    if number==2:
        return "2. love"
    if number==3:
        return "3. anger"
    if number==4:
        return "4. fear"
    if number==5:
        return "5. surprise"

    return "Not possible to identify"


def analyze_sentiment(phrase):
    tokens = preprocess_text(phrase)
    tokens_embeddings = doc2vec_text(tokens)

    # Reshape to have proper structure
    tokens_embeddings = np.array(tokens_embeddings).reshape(1, -1)

    # Predict the class using the SVM classifier
    predicted_class = svm_classifier.predict(tokens_embeddings)
    print(classify_emotion(predicted_class[0]))


In [ ]:
phrase = "i love love"
analyze_sentiment(phrase)

1. joy
